In [1]:
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
%matplotlib inline

import neuro_morpho_toolbox as nmt
pickle_in = open("/home/penglab/FeaCal/AP_dbscan_ns.pickle","rb")
example_ = pickle.load(pickle_in)
ns_xie = example_[0]

/home/penglab/anaconda3/lib/python3.7/site-packages/neuro_morpho_toolbox/
Loading CCF Atlas data...
Loading time: 0.86
Loading CCF brain structure data...
Loading time: 0.00


In [2]:
import ast
from scipy.spatial.distance import pdist, squareform
import numpy as np
import matplotlib as mpl
from sklearn.metrics import pairwise_distances
import matplotlib.pyplot as plt

def normalize(df, log=True):
    scaled_data = np.array(df) / np.sum(df, axis=0).values.reshape(-1,1) * 100000
    if log:
        scaled_data = np.log(scaled_data+100)
    scaled_dataDF = pd.DataFrame(scaled_data, index=df.index, columns=df.columns)
    return scaled_dataDF

def fromSOMAtoAXON(ns):
    somaDF = ns.features['soma_features'].region
    axonDF = ns.features['projection_features'].raw_data
    normaF=0

    somaregionDF = somaDF.copy()
    axonregionDF = axonDF.copy()      

    colname = ['Cluster Num','soma Region List','axon Region List']

    typeR, typeC = np.unique(ns.metadata['Cluster'] , return_counts = True)
    detailDF = pd.DataFrame(index = typeR,columns = ['swc list','soma region','max soma','total soma',
                                                             'max axon'])

        # for each cluster result UNDER EACH PARAMETER
    somaCLUSTERlist = []
    axonCLUSTERlist = []

    #axonCLUSTER = pd.DataFrame(index = axonregionDF.columns[:-1])
    for label_idx in detailDF.index:
        axonTEMPdf = pd.DataFrame(index = axonregionDF.columns)
        swcINlabel = ns.metadata[ns.metadata['Cluster']==label_idx]
        detailDF.loc[label_idx,'swc list'] = (swcINlabel.index).tolist()
        #assert(len(inter_list)==somaDF[somaDF['cluster'] !=label_idx].shape[0])
        detailDF.loc[label_idx,'soma region'] = swcINlabel['SomaRegion'].tolist()
        del_soma_region = detailDF.loc[label_idx,'soma region']
        while 'unknown' in del_soma_region:
            del_soma_region.remove('unknown')
            del_soma_region.append('others')
        while 'fiber tracts' in del_soma_region:
            del_soma_region.remove('fiber tracts')
            del_soma_region.append('others')
        IDrange, IDcounts = np.unique(del_soma_region, return_counts = True)
        #sort the counting
        count_sort_ind = np.argsort(-IDcounts)
        dicitem = []
        for i in range(1,min(len(IDrange)+1,100)):
            dicitem.append(str(i)+'somaR')
        for i in range(1,min(len(IDrange)+1,100)):
            dicitem.append(str(i)+'somaC')
        #select the first three regions
        dicvalue = IDrange[count_sort_ind][0:min(100,len(IDrange))]
        dicvalue = np.append(dicvalue,IDcounts[count_sort_ind][0:min(100,len(IDrange))])
        detailDF.loc[label_idx,'max soma'] = str(dict(zip(dicitem, dicvalue)))
        detailDF.loc[label_idx,'total soma'] = swcINlabel.shape[0]

        #axonLABEL stores all the rows under same cluster
        axonregionDF.loc[:,'cluster'] = ns.metadata['Cluster'].copy()
        axonLABEL = axonregionDF[axonregionDF['cluster']==label_idx].copy()
        axonLABEL.drop('cluster', axis=1,inplace=True)
        axonTEMPdf[label_idx]=axonLABEL.sum()
        if 'contra_fiber tracts' in axonTEMPdf.index:
            axonTEMPdf.drop(index=['contra_fiber tracts'],inplace = True)
        if 'ipsi_fiber tracts' in axonTEMPdf.index:
            axonTEMPdf.drop(index=['ipsi_fiber tracts'],inplace = True)
        if normaF:
            #normalize the sum
            axonTEMPdf = normalize(axonTEMPdf , log=True)
        axonTEMPdf.sort_values(by=label_idx, ascending=False, inplace=True)
        dicitem = []
        for i in range(1,min(10,sum(axonTEMPdf.loc[:,label_idx]>0))+1):
            dicitem.append(str(i)+'axonR')
        for i in range(1,min(10,sum(axonTEMPdf.loc[:,label_idx]>0))+1):
            dicitem.append(str(i)+'axonC')

        dicvalue = axonTEMPdf.index[:min(10,sum(axonTEMPdf.loc[:,label_idx]>0))-1]
        dicvalue = np.append(dicvalue, 'other')
        dicvalue = np.append(dicvalue,axonTEMPdf.loc[:,label_idx][:min(10,sum(axonTEMPdf.loc[:,label_idx]>0))-1])
        dicvalue = np.append(dicvalue,sum(axonTEMPdf.loc[:,label_idx][:-1])-sum(axonTEMPdf.loc[:,label_idx][:min(10,sum(axonTEMPdf.loc[:,label_idx]>0))-1]))
        detailDF.loc[label_idx,'max axon'] = str(dict(zip(dicitem, dicvalue)))
        axonCLUSTERlist.append(detailDF.loc[label_idx,'max axon'] )

    #(detailDF).to_csv('/home/penglab/Documents/detailXie.csv')
    clusterDF = detailDF.copy()
    dfname=[]
    for cluster_idx in clusterDF.index:
        dfname.append((str(cluster_idx) + 'DF'))

    #generate a dataframe for each cluster
    dicDF = {k:None for k in dfname}

    #iterate the correponding dataframe and cluster at the same time
    for key_iter,cluster_idx in zip(dicDF.keys(),clusterDF.index):
        dic_soma = ast.literal_eval(clusterDF.loc[cluster_idx,'max soma'])
        iterL = int(len(dic_soma.keys())/2)
        somaNAME = []
        somaCOUNT = []

        for i in range(1,int(iterL)+1):
            somaNAME.append(dic_soma[(str(i)+'somaR')])
        for i in range(1,iterL+1):
            somaCOUNT.append(dic_soma[str(i)+'somaC'])
        somaArr = np.array(somaCOUNT, dtype = float)
        somaArr = somaArr/sum(somaArr)
        dic_axon = ast.literal_eval(clusterDF.loc[cluster_idx,'max axon'])
        iterL = int(len(dic_axon.keys())/2)
        axonNAME = []
        axonCOUNT = []
        for i in range(1,iterL+1):
            axonNAME.append(dic_axon[(str(i)+'axonR')])
        for i in range(1,iterL+1):
            axonCOUNT.append(dic_axon[str(i)+'axonC']) 

        tempDF = pd.DataFrame()
        for row_soma in somaNAME:
            for col_axon,len_axon in zip(axonNAME,axonCOUNT):
                tempDF.loc[row_soma, col_axon] = len_axon

        assert(tempDF.shape[0] == len(somaArr))  
        tempDF=tempDF.astype(float)
        for i in range(tempDF.shape[0]):
            tempDF.iloc[i,:] = tempDF.iloc[i,:]*somaArr[i].copy()

        newDF = pd.DataFrame(columns=['from','to','value'],index=range(tempDF.shape[0]*tempDF.shape[1]))
        newDF.loc[:,'from'] = (tempDF.index).tolist()*len(tempDF.columns)
        newDF.loc[:,'to'] = [ele for ele in (tempDF.columns).tolist() for _ in range(len(tempDF.index))]
        valuelist=[]
        for iter_col in tempDF.columns:
            for iter_row in tempDF.index:
                valuelist.append(tempDF.loc[iter_row,iter_col])
        newDF.loc[:,'value']=valuelist
        dicDF[key_iter]= newDF.copy()
    return dicDF


In [3]:
dicXie = fromSOMAtoAXON(ns_xie)
for iter_key in dicXie.keys():
    tempDF = dicXie[iter_key]
    display(tempDF)
    #(tempDF).to_csv('/home/penglab/Documents/Sep12DBSCAN/'+str(iter_key)+'.csv')

,from,to,value
0,MOs,ipsi_CP,4.103907e+06
1,MOp,ipsi_CP,3.211753e+05
2,ORBl,ipsi_CP,2.141169e+05
3,FRP,ipsi_CP,1.784307e+05
4,PL,ipsi_CP,1.427446e+05
5,ACAd,ipsi_CP,7.137229e+04
6,ORBvl,ipsi_CP,3.568614e+04
7,MOs,contra_CP,2.497267e+06
8,MOp,contra_CP,1.954383e+05
9,ORBl,contra_CP,1.302922e+05


,from,to,value
0,DG,ipsi_CA3,58779.855609
1,DG,ipsi_DG,56707.086392
2,DG,other,612.434215


,from,to,value
0,DG,ipsi_DG,210775.055189
1,CA1,ipsi_DG,26346.881899
2,ENTl,ipsi_DG,13173.440949
3,DG,ipsi_CA1,133216.418082
4,CA1,ipsi_CA1,16652.052260
5,ENTl,ipsi_CA1,8326.026130
6,DG,ipsi_CA3,58495.943658
7,CA1,ipsi_CA3,7311.992957
8,ENTl,ipsi_CA3,3655.996479
9,DG,ipsi_CA2,10077.765275


,from,to,value
0,MOs,ipsi_MRN,189907.723760
1,MOp,ipsi_MRN,59970.860135
2,ACAd,ipsi_MRN,9995.143356
3,ORBl,ipsi_MRN,9995.143356
4,SSp-bfd,ipsi_MRN,9995.143356
5,SSp-m,ipsi_MRN,9995.143356
6,MOs,ipsi_CP,154909.960473
7,MOp,ipsi_CP,48918.934886
8,ACAd,ipsi_CP,8153.155814
9,ORBl,ipsi_CP,8153.155814


,from,to,value
0,VAL,ipsi_MOp,166473.005458
1,CL,ipsi_MOp,83236.502729
2,VM,ipsi_MOp,83236.502729
3,PO,ipsi_MOp,41618.251364
4,ZI,ipsi_MOp,41618.251364
5,ACAd,ipsi_MOp,20809.125682
6,AM,ipsi_MOp,20809.125682
7,MOp,ipsi_MOp,20809.125682
8,SMT,ipsi_MOp,20809.125682
9,VPL,ipsi_MOp,20809.125682


...

,from,to,value
0,MOs,ipsi_VAL,3.682696e+05
1,ACAd,ipsi_VAL,4.751866e+04
2,MOp,ipsi_VAL,2.969916e+04
3,others,ipsi_VAL,1.781950e+04
4,LHA,ipsi_VAL,1.187967e+04
5,FRP,ipsi_VAL,5.939833e+03
6,ORBvl,ipsi_VAL,5.939833e+03
7,PL,ipsi_VAL,5.939833e+03
8,RT,ipsi_VAL,5.939833e+03
9,VPL,ipsi_VAL,5.939833e+03


,from,to,value
0,SUB,ipsi_SUB,187041.213847
1,ProS,ipsi_SUB,46760.303462
2,others,ipsi_SUB,46760.303462
3,POST,ipsi_SUB,36369.124915
4,PRE,ipsi_SUB,36369.124915
5,DG,ipsi_SUB,25977.946368
6,CA1,ipsi_SUB,15586.767821
7,AV,ipsi_SUB,10391.178547
8,ENTm,ipsi_SUB,5195.589274
9,RE,ipsi_SUB,5195.589274


,from,to,value
0,SUB,ipsi_SUB,128057.937654
1,CA1,ipsi_SUB,59103.663533
2,ProS,ipsi_SUB,29551.831766
3,PRE,ipsi_SUB,19701.221178
4,others,ipsi_SUB,19701.221178
5,POST,ipsi_SUB,14775.915883
6,SUB,ipsi_AV,125459.161303
7,CA1,ipsi_AV,57904.228294
8,ProS,ipsi_AV,28952.114147
9,PRE,ipsi_AV,19301.409431


,from,to,value
0,MOs,ipsi_CP,79754.220267
1,MOp,ipsi_CP,63803.376213
2,others,ipsi_CP,55827.954187
3,VAL,ipsi_CP,23926.266080
4,PRE,ipsi_CP,23926.266080
5,VISp,ipsi_CP,15950.844053
6,DG,ipsi_CP,15950.844053
7,ACAv,ipsi_CP,15950.844053
8,ORBl,ipsi_CP,15950.844053
9,CA1,ipsi_CP,7975.422027


,from,to,value
0,others,ipsi_VISp,279158.098100
1,VISp,ipsi_VISp,114947.452159
2,VISl,ipsi_VISp,98526.387565
3,VISli,ipsi_VISp,16421.064594
4,VPL,ipsi_VISp,16421.064594
5,others,ipsi_VISl,108775.472307
6,VISp,ipsi_VISl,44789.900362
7,VISl,ipsi_VISl,38391.343167
8,VISli,ipsi_VISl,6398.557195
9,VPL,ipsi_VISl,6398.557195


In [4]:
dicXie

{'C1DF':      from            to         value
 0     MOs       ipsi_CP  4.103907e+06
 1     MOp       ipsi_CP  3.211753e+05
 2    ORBl       ipsi_CP  2.141169e+05
 3     FRP       ipsi_CP  1.784307e+05
 4      PL       ipsi_CP  1.427446e+05
 5    ACAd       ipsi_CP  7.137229e+04
 6   ORBvl       ipsi_CP  3.568614e+04
 7     MOs     contra_CP  2.497267e+06
 8     MOp     contra_CP  1.954383e+05
 9    ORBl     contra_CP  1.302922e+05
 10    FRP     contra_CP  1.085768e+05
 11     PL     contra_CP  8.686147e+04
 12   ACAd     contra_CP  4.343074e+04
 13  ORBvl     contra_CP  2.171537e+04
 14    MOs      ipsi_MOs  2.162321e+06
 15    MOp      ipsi_MOs  1.692251e+05
 16   ORBl      ipsi_MOs  1.128167e+05
 17    FRP      ipsi_MOs  9.401395e+04
 18     PL      ipsi_MOs  7.521116e+04
 19   ACAd      ipsi_MOs  3.760558e+04
 20  ORBvl      ipsi_MOs  1.880279e+04
 21    MOs    contra_MOs  1.769177e+06
 22    MOp    contra_MOs  1.384573e+05
 23   ORBl    contra_MOs  9.230489e+04
 24    FRP    con

In [9]:


swc_range6 = ['AA0022','AA0031','AA0052','AA0070','AA0071','AA0079','AA0088','AA0126','AA0173','AA0194',
              'AA0195','AA0199','AA0208','AA0227','AA0248','AA0251','AA0253','AA0257','AA0283','AA0375',
              'AA0381','AA0384','AA0387','AA0444','AA0468','AA0479','AA0489','AA0490','AA0491','AA0493',
              'AA0494','AA0496','AA0498','AA0500','AA0522','AA0523','AA0530','AA0535','AA0559','AA0560',
              'AA0562','AA0563','AA0564','AA0565','AA0567','AA0570','AA0611','AA0620','AA0681','AA0682',
              'AA0685','AA0688','AA0689','AA0696','AA0697','AA0703','AA0717','AA0720','AA0728','AA0731',
              'AA0804','AA0810','AA0813','AA0821','AA0822','AA0823','AA0826','AA0827','AA0828','AA0829',
              'AA0830','AA0831','AA0843','AA0860','AA0861','AA0875','AA0886','AA0894','AA0909','AA0912']
swc_range7 = ['AA0058','AA0124','AA0125','AA0139','AA0158','AA0161','AA0246','AA0247','AA0252','AA0306',
              'AA0380','AA0383','AA0423','AA0459','AA0476','AA0477','AA0478','AA0499','AA0532','AA0538',
              'AA0546','AA0551','AA0566','AA0569','AA0574','AA0613','AA0621','AA0631','AA0683','AA0684',
              'AA0687','AA0704','AA0706','AA0709','AA0721','AA0736','AA0811','AA0815','AA0816','AA0818',
              'AA0819','AA0820','AA0824','AA0825','AA0834','AA0847','AA0848','AA0849','AA0850','AA0851',
              'AA0855','AA0864','AA0885','AA0890','AA0910']

In [17]:
import os
import shutil
for swcname in swc_range6:
    oldname = '/home/penglab/Documents/Janelia_1000/'+str(swcname)+'.swc'
    newname = '/home/penglab/Documents/Ding_test/'+str(swcname)+'.swc'
    shutil.copyfile(oldname,newname)
for swcname in swc_range7:
    oldname = '/home/penglab/Documents/Janelia_1000/'+str(swcname)+'.swc'
    newname = '/home/penglab/Documents/Ding_test/'+str(swcname)+'.swc'
    shutil.copyfile(oldname,newname)

In [18]:
import neuro_morpho_toolbox as nmt
# ns=nmt.neuron_set('/Users/pengxie/Documents/Research/fullMorph_classification/data/registered_swc/Janelia_test/swc/Whole/')
ns=nmt.neuron_set('/home/penglab/Documents/Ding_test')


Loading...
number of files under swc_path: 135
100 loaded: 1.7s
Finding soma locations...
Getting projection features...
Number of input neurons: 135
Number of input features: 632
Getting dendrite features...
Number of input neurons: 135
Number of input features: 316
Getting metadata...


In [23]:
outputDF = pd.DataFrame(index = ns.features['projection_features'].raw_data.index, columns = ['max axon'])
for iter_index in ns.features['projection_features'].raw_data.index:
    tempDF = pd.DataFrame(index = ns.features['projection_features'].raw_data.columns, columns = [iter_index],data =ns.features['projection_features'].raw_data.loc[iter_index,:]).copy()
    tempDF.sort_values(by = tempDF.columns[0],ascending=False,inplace = True)
    for i in range(1,10):
        dicitem.append(str(i)+'axonR')
    for i in range(1,10):
        dicitem.append(str(i)+'axonC')
        
    dicvalue = tempDF.index[:9]
    dicvalue = np.append(dicvalue, 'other')
    dicvalue = np.append(dicvalue,axonTEMPdf.loc[:,label_idx][:3])
    dicvalue = np.append(dicvalue,sum(axonTEMPdf.loc[:,label_idx][:-1])-sum(axonTEMPdf.loc[:,label_idx][:3]))
    detailDF.loc[label_idx,'max axon'] = str(dict(zip(dicitem, dicvalue)))        
    outputDF.loc[iter_index,'max axon'] = 

,ipsi_FRP,ipsi_MOp,ipsi_MOs,ipsi_SSp-n,ipsi_SSp-bfd,ipsi_SSp-ll,ipsi_SSp-m,ipsi_SSp-ul,ipsi_SSp-tr,ipsi_SSp-un,...,contra_AN,contra_PRM,contra_COPY,contra_PFL,contra_FL,contra_FN,contra_IP,contra_DN,contra_VeCB,contra_fiber tracts
AA0022,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
AA0031,0.0,0.0,0.0,0.0,144.969861,35.002038,0.0,174.993852,0.000000,214.991717,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
AA0052,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,297.846888
AA0058,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
AA0070,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
AA0071,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
AA0079,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
AA0088,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
AA0124,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
AA0125,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [40]:
iter_index = 'AA0022'


tempDF = pd.DataFrame(index = ns.features['projection_features'].raw_data.columns, columns = [iter_index],data =ns.features['projection_features'].raw_data.loc[iter_index,:]).copy()
tempDF.sort_values(by = tempDF.columns[0],ascending=False,inplace = True)
tempDF

,AA0022
ipsi_RSPv,11178.053720
ipsi_CA1,10883.364132
ipsi_SUB,9492.661589
ipsi_POST,5324.001997
ipsi_DG,4761.123470
ipsi_fiber tracts,3191.335792
ipsi_CA3,2811.566428
ipsi_ProS,2230.863917
ipsi_PAR,1606.684320
ipsi_APr,1123.615065


In [39]:
tempDF.columns[0]

'AA0022'